# Lazy Evaluation

We will investigate lazy evaluation: an important and cool area where functional programming excels. We will demonstrate how laziness can be used to save time and space especially when we wish to process large amounts of data.

Let us start by examining the following function:

In [5]:
def func(x: Double, y: Double): Double = {
    println(s"Called: func($x, $y)")
    if (x >= 0){
        x*x - 2.5 * x + 4.3 * math.sin(x * 3.1415/180.0)
    } else {
        y
    }
}

val z = func(1.5, func(2.4, 1.2)) - func(-1.2, func(3.0, 2.5))

println(z)

Called: func(2.4, 1.2)
Called: func(1.5, -0.05993999644407877)
Called: func(3.0, 2.5)
Called: func(-1.2, 1.7250379807705825)
-3.1124804219960183


defined function func
z: Double = -3.1124804219960183

Notice that `func` has an if-then-else statement that compares `x` to 0. In particular, the value of `y` is needed in the else branch but never needed in the then branch.

Now consider the call to `func(1.5, func(2.4, 1.2))`.  Such a call would compute `x=1.5` and `y= func(2.4, 1.2)`. However, in this instance `x >= 0`, thus the value of `y` is not needed after all. 

However, for the call `func(-1.2, func(3.0, 2.5))`, we would need to know the value of the `y` argument for this call which is `func(3.0, 2.5)`.

Also, notice that the value of `z` is not really needed until we print it or use it in some calculation.

We will now present two interesting scala constructs that take advantage of these patterns.

## Call by Name

Call by name is a calling convention for functions that do not evaluate the arguments. Rather under call by name, the expression corresponding to the argument is passed to the function and evaluated when needed.

In Scala, function evaluation is normally performed using call by value.
However, we can use call by name as follows:

~~~
def multTwice(x :=> Int): Int
~~~

This signifies that a function multTwice takes in an argument `x`
which is an `Int`. However, it also signifies that `x` is passed using
"call by name". I.e, whenever we call this function

~~~
multTwice( expr )
~~~

the argument `expr` is passed unevaluated to the function. Every time `x` is used in the function, it is evaluated again.

In [17]:
def multTwice( x : => Int): Int = {
    // Note: the "=>" symbol in front of the type Int
    //    signifies call by name.
    x * x
}


def factorial(n: Int): Int = {
    println(s"Calling factorial $n")
    (1 to n).product
}


multTwice(factorial(5))

Calling factorial 5
Calling factorial 5


defined function multTwice
defined function factorial
res16_2: Int = 14400

Notice that in the example above there are two calls to factorial, why? This is because under call by name, the argument of the function `multTwice` is an unevaluated expression : `factorial(5)`. Every time, we access the argument `x`, it results in `factorial(5)` being called. If we do not want this behavior, then we force the argument to be evaluated as follows:

In [18]:
def multTwice( x : => Int): Int = {
    // Note: the "=>" symbol in front of the type Int
    //    signifies call by name.
    
    val z = x // This forces x to be evaluated and the result stored in z
    z * z
}

def factorial(n: Int): Int = {
    println(s"Calling factorial $n")
    (1 to n).product
}


multTwice(factorial(5))

Calling factorial 5


defined function multTwice
defined function factorial
res17_2: Int = 14400

Going back to the original example, we see how it can be achieved using call by name.

In [21]:
def func(x0: => Double, y0: => Double): Double = {
    // Call by name arguments.
   
    val x = x0 // Force x to be evaluatedd
    if (x >= 0){
        println(s"Called: func with x = $x, y = ???")
        x*x - 2.5 * x + 4.3 * math.sin(x * 3.1415/180.0)
    } else {
        val y = y0 // Force y to be evaluated
        println(s"Called: func with x = $x, y = $y")
        y
    }
}

val z = func(1.5, func(2.4, 1.2)) - func(-1.2, func(3.0, 2.5))

println(z)

Called: func with x = 1.5, y = ???
Called: func with x = 3.0, y = ???
Called: func with x = -1.2, y = 1.7250379807705825
-3.1124804219960183


defined function func
z: Double = -3.1124804219960183

Call by name is often used in context of lazy evaluation. Notice that for the function `func` above, we have to be careful when to evaluate the call by name parameters `x0` and `y0`. We can use a construct called lazy evaluation to simplify things considerably.

In [23]:
def func(x0: => Double, y0: => Double): Double = {
    // Call by name arguments.
   
    lazy val x = x0 // place x0 inside a "lazy" val x
    lazy val y = y0 // place y0 inside a "lazy" val y
    
    if (x >= 0){
        println(s"Called: func with x = $x, y = ???")
        x*x - 2.5 * x + 4.3 * math.sin(x * 3.1415/180.0)
    } else {
        println(s"Called: func with x = $x, y = $y")
        y
    }
}

lazy val z = func(1.5, func(2.4, 1.2)) - func(-1.2, func(3.0, 2.5))

println(z)

Called: func with x = 1.5, y = ???
Called: func with x = 3.0, y = ???
Called: func with x = -1.2, y = 1.7250379807705825
-3.1124804219960183


defined function func
z: Double = [lazy]

When we declare

~~~
lazy val x = x0
~~~

the right hand side expression `x0` is evaluated only when the value of `x` is used. Also, unlike call by name,  lazy values are "cached". I.e., the value of `x` once  evaluated now becomes a Double precision number.

In [30]:
def twice(x:Int): Int = {
    println(s"calling twice($x)")
    x + x
}

/*-- Eager evaluation -- what we are used to so far --*/
val y1 = twice(5) // Call happens at once
val y2 = twice(10) // call happens at once
println(y2)
println(y1)

/* lazy evaluation */
lazy val y = twice(10)
lazy val z = twice(20)
println(" Message z = "); println(z) // call to twice(20) here
println(" Message y = "); println(y) // call to twice(10) here
println(" Message z = "); println(z) // Does not call twice(20) again, reuses cached result.

calling twice(5)
calling twice(10)
20
10
 Message z = 
calling twice(20)
40
 Message y = 
calling twice(10)
20
 Message z = 
40


defined function twice
y1: Int = 10
y2: Int = 20
y: Int = [lazy]
z: Int = [lazy]

Lazy evaluation is useful because it saves unnecessary computation by evaluating arguments on demand. However, this is not the main use for lazy evaluation. The main reason we do lazy evaluation are two fold:
  - Creating functionality that is otherwise not possible.
  - Lazy data structures such as streams, iterators and so on.

### Unique Functionality provided by lazy evaluation

Lazy evaluation and call by name are very useful in designing embedded domain specific languages (DSLs) since they can enable useful functionality that cannot be provided otherwise.

In [76]:
/*-- my own whole loop --*/
def myWhileLoop (b : => Boolean) (body: => Unit) : Unit = {
    if (b) {
        body;
        myWhileLoop (b) (body)
    }   
}

defined function myWhileLoop

In [32]:
var i = 0

myWhileLoop (i <= 10) { 
                i = i + 1;
                println(i)
}

1
2
3
4
5
6
7
8
9
10
11


i: Int = 11

### Lazy Functional Data Structures

Scala supports lazy functional data structures that allow us to implement functionality that is otherwise quite cumbersome with eager evaluation methods.

### Streams

We will first study streams, a very powerful lazy abstraction.
At the first sight, a stream is very much like a list. However, 
elements are computed lazily as needed. We will illustrate this using an example.

First, we note that a stream is very much like a list.

In [84]:
val s1: Stream[Int] = Stream(1, 2, 3, 4, 5) 
// Stream can be converted to a list
val l1: List[Int]= s1.toList 
// Streams can support generic data types
val s2: Stream[String] = Stream("hello", "world", "this", "is", "CSCI3155")
// Head andd tail function
val h2: String= s2.head
// Take 3 elements from s2
val t2 = s2.take(3)
//Create a new stream that filters all elements of length = 5
val s3 = s2.filter( x => x.length == 5)

s1: Stream[Int] = Stream(1, 2, 3, 4, 5)
l1: List[Int] = List(1, 2, 3, 4, 5)
s2: Stream[String] = Stream("hello", "world", "this", "is", "CSCI3155")
h2: String = "hello"
t2: Stream[String] = Stream("hello", "world", "this")
s3: Stream[String] = Stream("hello", "world")

However, unlike a list, a stream is lazy. I.e, its elements are computed on demand.

Just like `::` is the Cons operator for a list in scala, we have
the operator `#::` that is a cons operator for a stream. However, `#::` does not evaluate its arguments. Read through the following example carefully.

In [34]:
def twice(x:Int): Int = {
    println(s"calling twice($x)")
    x + x
}

// Let us create a stream using the cons operator
lazy val myFirstStream: Stream[Int] = twice(10) #:: twice(20) #:: twice(30) #:: twice(40) #:: twice(50) #:: twice(60) #:: twice(70) #:: twice(80) #::Stream.empty
// Notice that twice(10) is called at this point
// But also printing the stream reveals that it is "unwound" or evaluated
// in a lazy manner
println(s"stream s = $myFirstStream")
val i1 = myFirstStream.head
println(i1)
val i2 = myFirstStream.tail.head // This will materialize the second element in the stream
println(i2)


calling twice(10)
stream s = Stream(20, ?)
20
calling twice(20)
40


defined function twice
myFirstStream: Stream[Int] = [lazy]
i1: Int = 20
i2: Int = 40

Notice in the above code that we accessed the first two elements of the stream. Therefore, `twice(10)` and `twice(20)` are called. However, the remaining elements of the stream are unevaluated because we have not tried to access them.

#### Memoization in Streams

It is important to note that a streams memoize in scala. I.e, once an element of a stream has been computed, its value is cached for future use.

Going back to the example above notice that `myFirstStream` is a stream whose first two elements are computed by calling the twice function. If we wanted to retrieve those elements again, twice function need not be called since their values are cached.

This is illustrated below by calling `take(2)` on the stream.
However, if we called `take(3)`, you will notice a new call to the twice function (`twice(30)`) to materialize the third element.

In [36]:
val firstTwo = myFirstStream.take(2).toList

val firstThree = myFirstStream.take(3).toList

calling twice(30)


firstTwo: List[Int] = List(20, 40)
firstThree: List[Int] = List(20, 40, 60)

### Stream Processing

Streams allow us to do some tasks efficiently that would not be otherwise possible. Let us say we wish to load a large text file and get  the first 10 lines that match some pattern of interests. The standard way would be:
  - Load the file from disk
  - Go through each line in the file
  - Match against the pattern
  - If it matches, print the line.
  - If more than 10 lines have been printed then exit.
  
  
Of course, we could avoid a loop and use functors to do this.

In [77]:
import scala.io.Source

var numFilterApps = 0

def applyMyFilter(s: String): Boolean = {
    numFilterApps = numFilterApps + 1 // Do some stats to count how many times this function calledd
    s.length >= 10 && !s.contains('.')
}

def processFileEager(f: String): List[String] = {
    Source.fromFile(f) // Load the file
          .getLines // Get the lines from the file
          .toList // Convert it to a list
          .filter(applyMyFilter)
          .take(10)
}

processFileEager("WarAndPeace.html")

println(s"Filter was called $numFilterApps times")

Filter was called 77788 times


import scala.io.Source


numFilterApps: Int = 27
defined function applyMyFilter
defined function processFileEager
res76_4: List[String] = List(
  "<!DOCTYPE html",
  "    <title>",
  "      War and Peace, by Leo Tolstoy",
  "    </title>",
  "    <style type=\"text/css\" xml:space=\"preserve\">",
  "    body { margin:5%; background:#faebd0; text-align:justify}",
  "    H1,H2,H3,H4,H5,H6 { text-align: center; margin-left: 15%; margin-right: 15%; }",
  "    hr  { width: 50%; text-align: center;}",
  "    blockquote {font-size: 97%; font-style: italic; margin-left: 10%; margin-right: 10%;}",
  "    pre     { font-style: italic; font-size: 90%; margin-left: 10%;}"
)

Notice that in the code above, although we output 10 lines in the end, the filter was run on the entire file with 77788 lines. 

If on the other hand, we used a stream, we will immediately see an advantage.

In [78]:
import scala.io.Source

numFilterApps = 0

def processFileLazy(f: String): List[String] = {
    Source.fromFile(f)
          .getLines
          .toStream // Convert it to a stream
          .filter(applyMyFilter)
          .take(10)
          .toList
}

processFileLazy("WarAndPeace.html")
println(s"Filter was called $numFilterApps times")

Filter was called 27 times


import scala.io.Source


defined function processFileLazy
res77_3: List[String] = List(
  "<!DOCTYPE html",
  "    <title>",
  "      War and Peace, by Leo Tolstoy",
  "    </title>",
  "    <style type=\"text/css\" xml:space=\"preserve\">",
  "    body { margin:5%; background:#faebd0; text-align:justify}",
  "    H1,H2,H3,H4,H5,H6 { text-align: center; margin-left: 15%; margin-right: 15%; }",
  "    hr  { width: 50%; text-align: center;}",
  "    blockquote {font-size: 97%; font-style: italic; margin-left: 10%; margin-right: 10%;}",
  "    pre     { font-style: italic; font-size: 90%; margin-left: 10%;}"
)

The reason we call filter method so few times is that a stream performs lazy processing.

### Infinite Streams

We can take things to a limit and create an infinite stream. 
Here is a seemingly non terminating recursive function. However, it does terminate instead of going on forever. Why?

In [20]:
def streamOfOnes: Stream[Double] = 1.0 #:: streamOfOnes

val s = streamOfOnes

defined function streamOfOnes
s: Stream[Double] = Stream(
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
...

In [24]:
def calculateContinuedFraction(s: Stream[Double], n: Int): Double = {
    if (n <= 0) { 1.0 }
    else {
        1.0 / (1.0 + calculateContinuedFraction(s.tail, n-1))
    }
}

calculateContinuedFraction(s, 100)

defined function calculateContinuedFraction
res23_1: Double = 0.6180339887498948

In [25]:
def fibonacciStream(x1: Int, x2: Int): Stream[Int] = x1 #:: fibonacciStream(x2, x1+x2)

defined function fibonacciStream

In [33]:
val allFibonacciNumbers:Stream[Int] = fibonacciStream(1,1)
/*-- List all odd fibonacci numbers <= 100000000 ? */
val oddFibBelowBillion = allFibonacciNumbers
    .filter( _ % 2 == 1)
    .takeWhile( _ <= 100000000)
    .toList

allFibonacciNumbers: Stream[Int] = Stream(
  1,
  1,
  2,
  3,
  5,
  8,
  13,
  21,
  34,
  55,
  89,
  144,
  233,
  377,
  610,
  987,
  1597,
  2584,
  4181,
  6765,
  10946,
  17711,
  28657,
  46368,
  75025,
  121393,
  196418,
  317811,
  514229,
  832040,
  1346269,
  2178309,
  3524578,
  5702887,
  9227465,
  14930352,
  24157817,
  39088169,
...
oddFibBelowBillion: List[Int] = List(
  1,
  1,
  3,
  5,
  13,
  21,
  55,
  89,
  233,
  377,
  987,
  1597,
  4181,
  6765,
  17711,
  28657,
  75025,
  121393,
  317811,
  514229,
  1346269,
  2178309,
  5702887,
  9227465,
  24157817,
  39088169
)

An alternative to a stream is a sequence or an iterator in scala.

## Iterators

Iterators are an alternative to streams that support lazy computation in an on-demand manner. We will illustrate iterators and show how they can be used in a similar manner as streams in scala.

An `Iterator` is a generic trait in scala that requires us to implement two methods `next` and `hasNext` as shown below.
 
  - Iterators extend from the trait called Iterator[T]
  - Iterators must implement two methods `hasNext` and `next`. 
    - `hasNext` returns a boolean that says if there are more elements
    - `next` returns the next element of type `T`.

In [79]:
class Range(start: Int, end: Int, step: Int = 1) extends Iterator[Int] {
    var current: Int = start
    assert(step > 0)
    assert(start <= end)
    
    override def hasNext: Boolean = current < end
    
    override def next: Int = {
        val retval = current
        current = current + step // Increment current
        retval
    }
}

val r1 = new Range(100, 142, 5)
val r1ToList = r1.toList // toList is inbuilt in the Iterator trait
// However, if we tried this again, we will not get anything
val r1ToListAgain = r1.toList

defined class Range
r1: Range = empty iterator
r1ToList: List[Int] = List(100, 105, 110, 115, 120, 125, 130, 135, 140)
r1ToListAgain: List[Int] = List()

However, the same functionality can be provided by a stream.

In [4]:
def streamRange(start: Int, end: Int, step: Int = 1): Stream[Int] = {
         if (start < end) 
            start #:: streamRange(start+step, end, step)
         else 
            Stream.empty
} 
val str1 = streamRange(100, 142, 5)
val str1ToList = str1.toList
// The nice thing about streams is we can iterate again
val str1ToListAgain = str1.toList

defined function streamRange
str1: Stream[Int] = Stream(100, 105, 110, 115, 120, 125, 130, 135, 140)
str1ToList: List[Int] = List(100, 105, 110, 115, 120, 125, 130, 135, 140)
str1ToListAgain: List[Int] = List(100, 105, 110, 115, 120, 125, 130, 135, 140)

Here is an iterator for Fibonacci numbers.

In [42]:
class AllFibonacci extends Iterator[Int] {
    var f1 = 1 // Iterators are different from streams in that they
    var f2 = 1 // must store a state.
    override def next: Int = {
        val tmp = f1 + f2
        f1 = f2
        f2 = tmp
        f1
    }
    
    override def hasNext: Boolean = {
        true
    }
}

defined class AllFibonacci

In [41]:
val fibValues= new AllFibonacci().take(1000).toList
val fibStrings = new AllFibonacci().take(1000).map( "Hello: " + _.toString).toList

fibValues: List[Int] = List(
  1,
  2,
  3,
  5,
  8,
  13,
  21,
  34,
  55,
  89,
  144,
  233,
  377,
  610,
  987,
  1597,
  2584,
  4181,
  6765,
  10946,
  17711,
  28657,
  46368,
  75025,
  121393,
  196418,
  317811,
  514229,
  832040,
  1346269,
  2178309,
  3524578,
  5702887,
  9227465,
  14930352,
  24157817,
  39088169,
  63245986,
...
fibStrings: List[String] = List(
  "Hello: 1",
  "Hello: 2",
  "Hello: 3",
  "Hello: 5",
  "Hello: 8",
  "Hello: 13",
  "Hello: 21",
  "Hello: 34",
  "Hello: 55",
  "Hello: 89",
  "Hello: 144",
  "Hello: 233",
  "Hello: 377",
  "Hello: 610",
  "Hello: 987",
  "Hello: 1597",
  "Hello: 2584",
  "Hello: 4181",
  "Hello: 6765",
  "Hello: 10946",
  "Hello: 17711",
  "Hello: 28657",
  "Hello: 46368",
  "Hello: 75025",
  "Hello: 121393",
  "Hello: 196418",
  "Hello: 317811",
  "Hello: 514229",
  "Hello: 832040",
  "Hello: 1346269",
  "Hello: 2178309",
  "Hello: 3524578",
  "Hello: 5702887",
  "Hello: 9227465",
  "Hello: 14930352",
  "Hello: 24157

Key differences between a stream and an iterator are:
  - A stream is immutable, we can iterate over a stream multiple times.
  - An iterator is typically mutable: we can usually iterate through it just once.
  - Streams support memoization whereas iterators do not.
  

## For Loops (Comprehensions)

Finally, we will revisit for loops and provide some useful ways to build iterators and streams.

We have seen for loops before, but thus far forbidden their use.

In [69]:
var i = 0
for (j <- 1 to 20) {
    i = i + j
}
println(s"sum = $i")

sum = 210


i: Int = 210

Note however, that a `for loop` in scala is not the same as in C++.
As an example, in C++, we would write the same for loop as

~~~
int i = 0;
int j = 0;
for (j = 0; j <= 20; ++j){
    i = i + j
}
~~~

However, the meaning of the for loop in C++ is different from that in Scala. For instance, let us take this for loop:

~~~
int j = 0;
for (j = 0; j <= 20; ++j){
    j = 0;
}
~~~

The body of the loop keeps assigning `j` to `0`. Such a loop would never terminate. However, let us do the same in Scala.


In [72]:
for (j <- 0 to 20) {
    j = 0
}

cmd72.sc:2: reassignment to val
    j = 0
      ^Compilation Failed

: 

Let us try a slightly different loop. 

In [80]:
var n = 10
for (j <- 0 to n) {
    n = n + 1
    println("Loop")
}
println("Hurray! Loop is complete!")

Loop
Loop
Loop
Loop
Loop
Loop
Loop
Loop
Loop
Loop
Loop
Hurray! Loop is complete!


n: Int = 21

In the first example, we see that  the loop index is treated as a `val` or an immutable while it clearly changes in each iteration. What is happening here?

In the second example, we are incrementing the upper bound in each loop iteration. Nevertheless, the loop runs `11` times and finishes.  What is happening here?

Note that a `for` loop is scala is rewritten as a functor as follows.

~~~
for (i <- iterator) {
   loop body 
}
~~~

is translated into

~~~
iterator.foreach ( i => { loop body } ) 
~~~

With this in mind, let us piece apart the two examples.  First, the loop index `i` is immutable since it is just translated into the formal parameter for an anonymous function.

Second, the loop  
~~~
for (j <- 0 to n) {
    n = n + 1
    println("Loop")
}
~~~

is transformed into 

~~~
val it = 0 to n
it.foreach ( j => {  
    n = n + 1
    println("Loop")
    } )
~~~

This the iterator `it` is created using the value of `n` at the beginning. Even though `n` changes, the iterator `it` does not change.

## For loops with yield

Scala allows us to do "comprehensions" using for loops and a `yield` keyword. The for-yield pattern looks like this

~~~
for (ident <- iterator) yield (expr)
~~~

It is translated into: `iterator.map( ident => { expr } ) ` 

The result of a for-yield loop is itself of the same type as the iterator. For instance, if the iterator is a `List` the result is an int.
If the iterator is a stream, then so is the result.


In [9]:
lazy val cons: Stream[(Int, Int)] = {
    for (i <- streamRange(0, 20, 3)) yield (i, i + 1)
} 
println(cons)

val cons2: List[(Int, String)]= {
    for (i <- List(0, 1, 2, 3, 4, 5)) yield (i, "Hello:"+i.toString)
}
println(cons2)

val cons3: Seq[(Int, String)] = {
    for (i <- 0 to 5) yield (i, "Hello:"+i.toString)
}

Stream((0,1), ?)
List((0,Hello:0), (1,Hello:1), (2,Hello:2), (3,Hello:3), (4,Hello:4), (5,Hello:5))


cons: Stream[(Int, Int)] = [lazy]
cons2: List[(Int, String)] = List(
  (0, "Hello:0"),
  (1, "Hello:1"),
  (2, "Hello:2"),
  (3, "Hello:3"),
  (4, "Hello:4"),
  (5, "Hello:5")
)
cons3: Seq[(Int, String)] = Vector(
  (0, "Hello:0"),
  (1, "Hello:1"),
  (2, "Hello:2"),
  (3, "Hello:3"),
  (4, "Hello:4"),
  (5, "Hello:5")
)

We can do nested iterations in the same for loop


In [13]:
for ( i <- (1 to 5);
      j <- (i to i+25)
    ) yield (i, j)

res12: collection.immutable.IndexedSeq[(Int, Int)] = Vector(
  (1, 1),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (1, 6),
  (1, 7),
  (1, 8),
  (1, 9),
  (1, 10),
  (1, 11),
  (1, 12),
  (1, 13),
  (1, 14),
  (1, 15),
  (1, 16),
  (1, 17),
  (1, 18),
  (1, 19),
  (1, 20),
  (1, 21),
  (1, 22),
  (1, 23),
  (1, 24),
  (1, 25),
  (1, 26),
  (2, 2),
  (2, 3),
  (2, 4),
  (2, 5),
  (2, 6),
  (2, 7),
  (2, 8),
  (2, 9),
  (2, 10),
  (2, 11),
  (2, 12),
  (2, 13),
...

We can write if conditions to guard which iterations pass through to the yield statement.

In [16]:
for ( i <- (1 to 5);
      j <- (i to i+25);
      if ((i+j) % 5 == 4); // Filter applied to pairs (i,j)
      k <- (i+j to i+j+5) // (i,j) have the filter applied
    ) yield (k)

res15: collection.immutable.IndexedSeq[Int] = Vector(
  4,
  5,
  6,
  7,
  8,
  9,
  9,
  10,
  11,
  12,
  13,
  14,
  14,
  15,
  16,
  17,
  18,
  19,
  19,
  20,
  21,
  22,
  23,
  24,
  24,
  25,
  26,
  27,
  28,
  29,
  4,
  5,
  6,
  7,
  8,
  9,
  9,
  10,
...

Here is an example that prints out all pythagorean triples

In [19]:
var counter = 0

def isPerfectSquare(s: Int): Boolean = {
    counter = counter + 1
    val f = s.toDouble
    val d = math.sqrt(f)
    val sqrt = d.toInt
    sqrt * sqrt == s || (sqrt +1 )* (sqrt + 1) == s
}

def getPythagoreanTriples(n: Int) = 
    for (j <- (1 to n-1).toStream;
         k <- (j+1 to n).toStream;
            if (isPerfectSquare(j * j + k * k))
        ) yield (j,k)

println(getPythagoreanTriples(100).take(10).toList)

List((3,4), (5,12), (6,8), (7,24), (8,15), (9,12), (9,40), (10,24), (11,60), (12,16))


counter: Int = 1038
defined function isPerfectSquare
defined function getPythagoreanTriples

More readings: 
  - Functional Programming in Scala (https://www.manning.com/books/functional-programming-in-scala)
  - Haskell programming language
  http://learnyouahaskell.com/